In [1]:
!pip install -q "flash-attn>=2.4.2.post1"  # CUDA 12용 / Colab A100
# !pip install -q transformers accelerate bitsandbytes
!pip install -q auto-gptq>=0.8.0 optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.2 MB/s eta 0:00:00


In [2]:
!pip uninstall -y flash-attn
!rm -rf /usr/local/lib/python3.11/dist-packages/flash_attn*
!rm -rf /root/.cache/pip



Found existing installation: flash_attn 2.8.0.post2
Uninstalling flash_attn-2.8.0.post2:
  Successfully uninstalled flash_attn-2.8.0.post2


In [3]:
!pip uninstall -y transformers
!pip install transformers==4.41.0

Found existing installation: transformers 4.53.0
Uninstalling transformers-4.53.0:
  Successfully uninstalled transformers-4.53.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 77.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2


In [4]:
!pip install -U transformers==4.41.0 peft==0.15.2

In [6]:
MODEL_CHECKPOINT_PATH = "/content/drive/MyDrive/9. Lab/BARQA/Packaging & Download/atsc/googleflan-t5-large-SA_E2E_250616_ver1/checkpoint-2496"
INPUT_PATH_1 = "/content/implicit_case.xlsx"
INPUT_PATH_2 = "/content/translated_crawl_results.xlsx"
OUTPUT_PATH = "/content/Aspect_Extraction_implicit.xlsx"

In [ ]:
# %cd /content

In [9]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
!pip install --upgrade transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 97.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.0
    Uninstalling transformers-4.41.0:
      Successfully uninstalled transformers-4.41.0


In [ ]:
# ==============================================================
# 0. Installation (Colab A100) – Skip if already installed
# !pip install -q auto-gptq>=0.8.0 optimum "flash-attn>=2.4.2.post1wjs"
# ==============================================================

# ------------------ 1. Modules & Environment Setup ----------------------
import re, yaml, time
from pathlib import Path
from typing import Dict, List, Tuple

import pandas as pd
import torch
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

INPUT_CSV      = f"{INPUT_PATH_1}"
CONTEXT_XLSX   = f"{INPUT_PATH_2}"
CONTEXT_SHEET  = ""
BATCH_SIZE     = 32
OUTPUT_NAME    = f"{OUTPUT_PATH}"
BRAND_YML = Path("/content/drive/MyDrive/9. Lab/BARQA/Packaging & Download/Data/brands_0519.yaml")
ASPT_YML  = Path("/content/drive/MyDrive/9. Lab/BARQA/Packaging & Download/Data/aspects_0519.yaml")

MODEL_ID      = "TheBloke/OpenHermes-2.5-Mistral-7B-GPTQ"  # 4-bit quantized model
MAX_INPUT     = 8192  # Llama 3 8B context window
MAX_NEW       = 32
DEVICE        = "cuda" if torch.cuda.is_available() else "cpu"

# ------------------ 2. Load GPTQ Model ------------------------
print("🔧 Loading GPTQ model …")
model = AutoGPTQForCausalLM.from_quantized(
    MODEL_ID,
    device_map="auto",
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=True,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
model = torch.compile(model)
model.eval()
print("✅ Model ready.")

# ------------------ 3. Load Data ---------------------------
print("📄 Loading context …")
ctx_df = pd.read_excel(CONTEXT_XLSX)
conversation_list = ctx_df["Conversation Stream"].tolist()
sent2idx = {s: i for i, s in enumerate(conversation_list)}
print(f"✅ Context rows: {len(conversation_list):,}")

print("📄 Loading test csv …")
test_df = pd.read_excel(INPUT_CSV)
print(f"✅ Test rows   : {len(test_df):,}")

# =============================================================
# 4. YAML → Regex → BrandClassifier  ────────────────────────
def _load_yaml(path: Path) -> Dict[str, str]:
    with path.open(encoding="utf-8") as f:
        return yaml.safe_load(f)

def _compile_dict(src: Dict[str, str], flags: int = 0) -> Dict[str, re.Pattern]:
    return {k: re.compile(v, flags) for k, v in src.items()}

class BrandClassifier:
    """Regex-based classifier with LLM fallback"""

    APPLE_REQ_ASP = {
        'Camera General', 'Design General', 'Chipset General',
        'Sustainability General', 'Portrait Studio',
        'On-device', 'Grip', "Thin", "AI General"
    }

    def __init__(self) -> None:
        brand_src  = _load_yaml(BRAND_YML)
        aspect_src = _load_yaml(ASPT_YML)
        self.brand_rx  = _compile_dict(brand_src)
        self.aspect_rx = _compile_dict(aspect_src)
        self._apple_kw_rx = re.compile(
            r"(?i)\b(?:apple|apple's|iphone|아이폰|애플)\b"
        )

    def classify(self, text: str, used_llm: bool = False) -> Tuple[str, str]:
        # 1) Try brand first
        for brand, brx in self.brand_rx.items():
            if brx.search(text):
                if brand == "Apple":
                    for asp, arx in self.aspect_rx.items():
                        if arx.search(text):
                            return "Apple", asp
                return brand, brand

        # 2) No brand → check aspect
        for asp, arx in self.aspect_rx.items():
            if arx.search(text):
                if asp not in self.APPLE_REQ_ASP:
                    return "Apple", asp
                if self._apple_kw_rx.search(text):
                    return "Apple", asp
                return "Unknown", "brand_required_aspect_detected"

        # 3) Nothing matched
        return "Unknown", "general"

clf = BrandClassifier()

# ------------------ 6. LLM Prompt Template -------------------------
SYS = "You are an intelligent assistant."
def _prompt(ctx: str, sent: str) -> str:
    return (
        f"{SYS}\nGiven a context, identify what the target sentence is describing. "
        "Answer with one noun phrase only.\n"
        """        [Example 1]
        <Context>
        Flagship phones with great cameras have been IP-rated for a few years now.
        I have had an Apple iPhone 15 Pro with me for a while now, but I never truly checked how effective the IP rating is except for a few splashes and washes here and there.
        It gave me more confidence to take it along for walks even if it drizzled or light rains and have done so without much thought.
        <Sentence>
        It gave me more confidence to take it along for walks even if it drizzled or light rains and have done so without much thought.
        <Answer>
        IP rating of iPhone 15 Pro

        [Example 2]
        <Context>
        Nova Earbuds come with noise cancellation and customizable touch controls.
        - Intuitive Touch Control: Simple tap to play or pause music.
        <Sentence>
        - Intuitive Touch Control: Simple tap to play or pause music.
        <Answer>
        Nova Earbuds"""
        f"<Context>\n{ctx}\n<Sentence>\n{sent}\n<Question>What is the subject of this sentence given a context?\nAnswer:"
    )

# ------------------ 7. LLM Inference -----------------------------
def llm_batch(sent_list: List[str]) -> Tuple[List[str], List[str]]:
    prompts, ctx_list = [], []
    for s in sent_list:
        i   = sent2idx.get(s, -1)
        ctx = " ".join(conversation_list[max(0, i-3):i]) if i >= 0 else ""
        ctx_list.append(ctx)
        prompts.append(_prompt(ctx, s))

    toks = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_INPUT,
    ).to(DEVICE)

    with torch.no_grad():
        out = model.generate(
            **toks,
            max_new_tokens=MAX_NEW,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False,
        )
    dec = tokenizer.batch_decode(out, skip_special_tokens=True)
    return [d.split("Answer:")[-1].strip() for d in dec], ctx_list

# ------------------ 9. Batch Classification Logic ------------------------
def classify_batch(texts: List[str]) -> List[Tuple[str, str]]:
    """Apply regex first → fallback to LLM if unknown"""
    results: List[Tuple[str, str]] = []
    llm_raw: List[str]             = [""] * len(texts)
    llm_ctx: List[str]             = [""] * len(texts)

    pending_texts, idx_map = [], []

    # 1) Regex-based classification
    for i, t in enumerate(texts):
        lab = clf.classify(t)
        if lab[0] == "Unknown":
            pending_texts.append(t); idx_map.append(i)
            results.append(lab)
        else:
            results.append(lab)

    # 2) Call LLM only for unknowns
    if pending_texts:
        llm_ans, ctx_list = llm_batch(pending_texts)
        for ii, ans, ctx in zip(idx_map, llm_ans, ctx_list):
            llm_raw[ii] = ans
            llm_ctx[ii] = ctx
            parsed = clf.classify(f"{texts[ii]} (about {ans})", used_llm=True)
            results[ii] = parsed if parsed[0] != "Unknown" else ("Unknown", f"about {ans}")

    return results, llm_raw, llm_ctx

# ------------------ 10. Run Classification ----------------------------
from tqdm.notebook import tqdm  # Make sure this is imported at the top

def run(df: pd.DataFrame) -> pd.DataFrame:
    sents = df["Conversation Stream"].tolist()
    preds, llm_texts, ctx_list = [], [], []

    print(f"🚀 Total sentences to process: {len(sents):,}")
    for i in tqdm(range(0, len(sents), BATCH_SIZE), desc="🧠 Processing", unit="batch"):
        p_batch, a_batch, c_batch = classify_batch(sents[i : i + BATCH_SIZE])
        preds.extend(p_batch)
        llm_texts.extend(a_batch)
        ctx_list.extend(c_batch)

    out = df.copy()
    out["Predicted_Brand"], out["Predicted_Aspect"] = zip(*preds)
    out["LLM Raw Answer"] = llm_texts
    out["LLM Context"]    = ctx_list
    return out

# ------------------ 11. Save Results --------------------------------
from zoneinfo import ZoneInfo  # Python 3.9+
t0 = time.time()
result_df = run(test_df)
print(f"🏁 Completed in {time.time()-t0:.1f}s  ({len(result_df):,} rows)")

out_path = f"{OUTPUT_NAME}"
result_df.to_excel(out_path, index=False)


WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.


🔧 Loading GPTQ model …


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.
WARNING - MistralGPTQForCausalLM hasn't fused attention module yet, will skip inject fused attention.


✅ Model ready.
📄 Loading context …
✅ Context rows: 24,579
📄 Loading test csv …
✅ Test rows   : 22,248
🚀 Total sentences to process: 22,248


🧠 Processing:   0%|          | 0/696 [00:00<?, ?batch/s]

🏁 Completed in 5692.8s  (22,248 rows)
